In [63]:
"""numpy and torch"""
import numpy as np
import torch

"""PIL"""
from PIL import Image

"""torchvision and utils"""
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
from torchvision.utils import save_image

"""os"""
import os


"""Get image to tensor"""
transform = transforms.Compose([
    transforms.PILToTensor()
])

"""Loading data into arrays"""
xtrain, xtrain, xtest, ytest = [], [], [], []

"""training data"""
trainDIRs = ['../../../AD_NC/train/AD/', '../../../AD_NC/train/NC']
size = [0, 0]
for i, DIR in enumerate(trainDIRs):
    for filename in os.listdir(DIR):
        f = os.path.join(DIR, filename)
        img = Image.open(f)
        tensor = transform(img).float()
        tensor.require_grad = True
        xtrain.append(tensor/255)
        size[i] += 1
xtrain = torch.stack(xtrain)
ytrain = torch.from_numpy(np.concatenate((np.ones(size[0]), np.zeros(size[1])), axis=0))

"""testing data"""
testDIRs = ['../../../AD_NC/test/AD/', '../../../AD_NC/test/NC']
size = [0, 0]
for i, DIR in enumerate(testDIRs):
    for filename in os.listdir(DIR):
        f = os.path.join(DIR, filename)
        img = Image.open(f)
        tensor = transform(img).float()
        tensor.require_grad = True
        xtest.append(tensor/255)
        size[i] += 1
xtest = torch.stack(xtest)
ytest = torch.from_numpy(np.concatenate((np.ones(size[0]), np.zeros(size[1])), axis=0))

In [79]:
def posEmbedding(imgs, wsize, hsize):
    N, C, W, H = imgs.shape #number imgs, channels, width, height
    size = (N, C, W // wsize, wsize, H // hsize, hsize)
    perm = (0, 2, 4, 1, 3, 5) #bring col, row index of patch to front
    flat = (1, 2) #flatten (col, row) index into col*row entry index for patches
    imgs = imgs.reshape(size).permute(perm).flatten(*flat)
    return imgs

In [80]:
patches = posEmbedding(xtrain, 16, 16)